# Weather trader
This was a simple first pass, creating a weather niceness score and looking at the correlation between it and the S & P

In [8]:
import vectorbt as vbt
import numpy as np
import pandas as pd
import nbformat
import datetime as dt
import scipy.stats as stats
import plotly.graph_objects as go

# General idea
1. Get data from wherever + put it into (pd or np?)
2. Add whatever indicator columns you need to buy/sell
3. Add buy/sell columns

[Ejemplo uno](https://quantnomad.com/implementing-ut-bot-strategy-in-python-with-vectorbt/)

[Ejemplo dos](https://vectorbt.pro/tutorials/basic-rsi-strategy/#single-backtest)

In [9]:
spy = vbt.YFData.download('SPY')

In [10]:
df = pd.DataFrame(spy.data['SPY'])
df.index = pd.Series(df.index).dt.tz_localize(None)
df.index

DatetimeIndex(['1993-01-29 05:00:00', '1993-02-01 05:00:00',
               '1993-02-02 05:00:00', '1993-02-03 05:00:00',
               '1993-02-04 05:00:00', '1993-02-05 05:00:00',
               '1993-02-08 05:00:00', '1993-02-09 05:00:00',
               '1993-02-10 05:00:00', '1993-02-11 05:00:00',
               ...
               '2022-12-09 05:00:00', '2022-12-12 05:00:00',
               '2022-12-13 05:00:00', '2022-12-14 05:00:00',
               '2022-12-15 05:00:00', '2022-12-16 05:00:00',
               '2022-12-19 05:00:00', '2022-12-20 05:00:00',
               '2022-12-21 05:00:00', '2022-12-22 05:00:00'],
              dtype='datetime64[ns]', name='Date', length=7531, freq=None)

In [11]:
# import weather_hist into a dataframe
weather_hist = pd.read_csv('data/weather_hist.csv', index_col=0) 
weather_hist.index = pd.to_datetime(weather_hist.index)

In [12]:
# join weather_hist with df on date
df = df.join(weather_hist, how='inner',on='Date', lsuffix='_df', rsuffix='_weather_hist')

In [13]:
# rename apparent_temperature to temp
df.rename(columns={
  'apparent_temperature (°F)':'temp',
  'precipitation (mm)':'precip',
  'cloudcover (%)':'clouds',
  }, inplace=True)

In [14]:
def weather_score(row):
    # Old:
    # temp = min(abs(row['temp'] - 72.0001) ** -1, 50)
    # precip = -15 * row['precip'] ** 0.25
    # return 0.5 * (100 - row['clouds']) + temp + precip
    temp = stats.norm.pdf(row['temp'], 72, 12.5) * 1500
    precip = -15 * row['precip'] ** 0.25
    return 0.5 * (100 - row['clouds']) + temp + precip
# how to optimize weather score? optimize for what nd when?

# generate column from lambda function over rows
df['niceness'] = df.apply(lambda row: weather_score(row), axis=1)
# df['niceness'] = df.apply(lambda row: weather_score(row['temp'], row['clouds'], row['precip']), axis=1)

In [15]:
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,temp,precip,clouds,niceness
Date,,,,,,,,,,,
2010-01-04 05:00:00,87.704920,88.501029,87.033689,88.454201,118944600,0.0,0,10.5,0.0,27,36.500265
2010-01-05 05:00:00,88.399558,88.727367,88.079549,88.688339,111579900,0.0,0,12.9,0.0,3,48.500670
2010-01-06 05:00:00,88.602481,88.969318,88.532239,88.750778,116074400,0.0,0,15.6,0.0,36,32.001817
2010-01-07 05:00:00,88.586872,89.234689,88.337112,89.125420,131091100,0.0,0,18.6,0.0,4,48.005214
2010-01-08 05:00:00,88.891271,89.461040,88.711759,89.422012,126402800,0.0,0,22.8,0.0,100,0.020703


In [16]:
# generate new column from lambda function over rows based on previous row
df['delta_nice'] = df.niceness.diff()

In [17]:
# add buy column with true if delta_nice > 0
df['sell'] = df.delta_nice > 0
df['buy'] = df.delta_nice < 0
# df = df[df.index.year == 2016]
# create portfolio and run backtest
portfolio = vbt.Portfolio.from_signals(df.Close, df.buy, df.sell, freq='D', )
portfolio.stats()

Start                               2010-01-04 05:00:00
End                                 2019-12-31 05:00:00
Period                               2516 days 00:00:00
Start Value                                       100.0
End Value                                    271.746338
Total Return [%]                             171.746338
Benchmark Return [%]                         246.875025
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              12.924577
Max Drawdown Duration                 168 days 00:00:00
Total Trades                                        792
Total Closed Trades                                 792
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  60.984848
Best Trade [%]                                 6.407876
Worst Trade [%]                               -7

In [18]:
portfolio.alpha()

ayo


0.055254062042598306

In [19]:
portfolio.plot().show()

In [ ]:
# portfolio.returns()
# benchmark_returns = pd.DataFrame(spy.data['SPY']).vbt.returns()
# # portfolio.returns().vbt.beta(benchmark_rets=benchmark_returns)
# rets = spy.data['SPY'].pct_change()
# ret_acc = rets.vbt.returns(freq='d')
# ret_acc.beta(benchmark_rets=benchmark_returns)
# # portfolio.vbt.returns.beta(
# #     benchmark_rets=benchmark_returns,
# #     # wrap_kwargs=None
# # )

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
non-precise type readonly array(pyobject, 2d, A)
During: typing of argument at /Users/johnboesen/Documents/Code/#Algotraders/yja/yja-env/lib/python3.9/site-packages/vectorbt/returns/nb.py (507)

File "yja-env/lib/python3.9/site-packages/vectorbt/returns/nb.py", line 507:
def beta_nb(returns: tp.Array2d, benchmark_rets: tp.Array2d) -> tp.Array1d:
    <source elided>
    """2-dim version of `beta_1d_nb`."""
    out = np.empty(returns.shape[1], dtype=np.float_)
    ^


In [29]:
# df = df[df.index.year]
num = 10
entry_points = np.linspace(-20,24, num=num)
exit_points = np.linspace(25,55, num=num)
returns_mtx = pd.DataFrame(columns=entry_points, index=exit_points)
for entry in entry_points:
  for exit in exit_points:
    df['buy'] = df.delta_nice < entry
    df['sell'] = df.delta_nice > exit
    portfolio = vbt.Portfolio.from_signals(df.Close, df.buy, df.sell, freq='D')
    returns_mtx[entry][exit] = portfolio.total_return()

In [30]:
returns_mtx.vbt.heatmap(
        xaxis_title = "entry",
        yaxis_title="exit").show()